In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install auto-sklearn
import autosklearn.classification
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
!pip install catboost
from catboost import CatBoostClassifier
from sklearn import svm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
train = pd.read_csv('/content/drive/MyDrive/cs_test/train.csv')
train

,map_id,team1_id,team2_id,map_name,who_win
0,289,6665,7718,Ancient,0
1,715,4411,10577,Inferno,0
2,157,11251,9455,Nuke,1
3,524,4608,7532,Mirage,0
4,404,8637,6667,Overpass,1
...,...,...,...,...,...
708,709,6667,4773,Inferno,0
709,528,9215,5995,Ancient,1
710,163,4869,9565,Mirage,1
711,96,10426,4991,Nuke,1


In [3]:
test = pd.read_csv('/content/drive/MyDrive/cs_test/test.csv')
test.drop(columns='index', inplace=True)
print(len(test))

30


In [4]:
player_feats = pd.read_csv('/content/drive/MyDrive/cs_test/players_feats.csv')
player_feats['team1_id'] = player_feats['team_id']
player_feats

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id,team1_id
0,4954,90,42.2,112,0.80,76.3,5.9,6,156,0.58,...,25,12,2.08,1.28,84.0,25.0,6665,Ancient,635,6665
1,5794,45,60.0,57,0.79,82.3,10.9,3,68,0.66,...,54,34,1.59,1.17,70.4,16.7,7532,Ancient,635,7532
2,4954,156,51.9,167,0.93,63.5,3.4,10,265,0.59,...,26,19,1.37,1.10,88.5,20.5,6665,Dust2,583,6665
3,5794,449,53.5,427,1.05,86.7,13.1,23,618,0.73,...,62,49,1.27,1.10,79.0,17.4,7532,Dust2,583,7532
4,7998,173,32.9,130,1.33,82.4,2.9,9,225,0.77,...,27,25,1.08,1.08,81.5,16.2,4608,Dust2,439,4608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,9031,69,49.3,70,0.99,69.8,5.7,4,114,0.61,...,16,18,0.89,1.11,75.0,21.4,10503,Ancient,284,10503
1482,7716,104,55.8,113,0.92,71.6,5.9,6,172,0.60,...,33,34,0.97,1.30,75.8,28.4,7020,Vertigo,27,7020
1483,12521,141,37.6,159,0.89,79.7,7.4,8,210,0.67,...,9,15,0.60,0.77,55.6,4.3,8297,Vertigo,27,8297
1484,7716,155,58.7,154,1.01,71.1,7.9,10,246,0.63,...,30,17,1.76,1.39,83.3,22.9,7020,Ancient,237,7020


In [5]:
df = pd.concat([train, test], ignore_index=True)
df = pd.merge(df, player_feats, on = ['map_id', 'team1_id'])
df

,map_id,team1_id,team2_id,map_name_x,who_win,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,...,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name_y
0,289,6665,7718,Ancient,0.0,4954,258,36.0,293,0.88,...,210,55,55,33,1.67,1.18,83.6,19.7,6665,Ancient
1,715,4411,10577,Inferno,0.0,8611,178,39.3,208,0.86,...,186,-23,47,49,0.96,1.01,74.5,16.3,4411,Inferno
2,157,11251,9455,Nuke,1.0,7938,494,52.8,397,1.24,...,150,8,11,21,0.52,0.73,81.8,5.0,11251,Nuke
3,524,4608,7532,Mirage,0.0,7998,474,29.1,304,1.56,...,261,66,74,57,1.30,1.18,81.1,20.3,4608,Mirage
4,404,8637,6667,Overpass,1.0,2898,217,55.8,248,0.88,...,78,11,20,17,1.18,1.09,80.0,17.0,8637,Overpass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,515,4608,6667,Inferno,NaN,7998,323,48.0,239,1.35,...,157,12,33,44,0.75,0.92,69.7,11.7,4608,Inferno
739,201,4608,6667,Overpass,NaN,7998,196,39.3,119,1.65,...,114,42,38,23,1.65,1.35,76.3,21.3,4608,Overpass
740,674,4608,6667,Ancient,NaN,7998,162,36.4,128,1.27,...,100,49,14,12,1.17,0.94,92.9,11.9,4608,Ancient
741,357,4608,6667,Mirage,NaN,7998,574,32.6,379,1.51,...,317,68,93,72,1.29,1.19,75.3,19.1,4608,Mirage


In [6]:
player_feats['team2_id'] = player_feats['team_id']
df_train = pd.merge(df, player_feats, on = ['map_id', 'team2_id'])
df_train

,map_id,team1_id_x,team2_id,map_name_x,who_win,p1_id_x,p1_total_kills_x,p1_headshots_x,p1_total_deaths_x,p1_kd_ratio_x,...,p5_kill_death_difference_y,p5_total_opening_kills_y,p5_total_opening_deaths_y,p5_opening_kill_ratio_y,p5_opening_kill_rating_y,p5_team_win_percent_after_first_kill_y,p5_first_kill_in_won_rounds_y,team_id_y,map_name,team1_id_y
0,289,6665,7718,Ancient,0.0,4954,258,36.0,293,0.88,...,-34,21,20,1.05,1.06,76.2,19.0,7718,Ancient,7718
1,715,4411,10577,Inferno,0.0,8611,178,39.3,208,0.86,...,90,64,25,2.56,1.39,75.0,23.3,10577,Inferno,10577
2,157,11251,9455,Nuke,1.0,7938,494,52.8,397,1.24,...,-32,32,35,0.91,1.09,53.1,15.5,9455,Nuke,9455
3,524,4608,7532,Mirage,0.0,7998,474,29.1,304,1.56,...,-13,15,21,0.71,0.82,73.3,7.9,7532,Mirage,7532
4,404,8637,6667,Overpass,1.0,2898,217,55.8,248,0.88,...,26,20,10,2.00,1.01,80.0,12.4,6667,Overpass,6667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,515,4608,6667,Inferno,NaN,7998,323,48.0,239,1.35,...,139,57,32,1.78,1.01,68.4,10.7,6667,Inferno,6667
739,201,4608,6667,Overpass,NaN,7998,196,39.3,119,1.65,...,91,46,22,2.09,1.08,80.4,15.0,6667,Overpass,6667
740,674,4608,6667,Ancient,NaN,7998,162,36.4,128,1.27,...,62,20,20,1.00,0.88,80.0,8.9,6667,Ancient,6667
741,357,4608,6667,Mirage,NaN,7998,574,32.6,379,1.51,...,32,53,26,2.04,1.05,66.0,11.8,6667,Mirage,6667


In [7]:
#дропнул бесполезные колонки, пропуски заполнил нулями, тк скорее всего они значат, что тот показатель игрока нулевой
features = df_train.drop(['map_id', 'who_win','map_name_x', 'map_name_y', 'team_id_x', 'team_id_y', 'team1_id_y'], axis=1).fillna(0)
features['map_name'] = pd.factorize(features['map_name'])[0].astype(int)
train_target = df_train['who_win'].iloc[:713].astype(int)
train_features=features.iloc[:713]
test_features=features.iloc[713:]

In [8]:
scaler = MinMaxScaler()
train_features=scaler.fit_transform(train_features)
test_features=scaler.transform(test_features)

In [9]:
model = LogisticRegression(max_iter=10000, random_state=42).fit(train_features, train_target)

In [10]:
print(cross_val_score(model, train_features, train_target, cv=5).mean())

0.5988673298532453


In [11]:
model = CatBoostClassifier(iterations=1000,
                           task_type="GPU",
                           devices='0:1', verbose=0)
print(cross_val_score(model, train_features, train_target, cv=5).mean())

0.5553727962178666


In [12]:
model = svm.SVC()
print(cross_val_score(model, train_features, train_target, cv=5).mean())

0.5890475721461638


In [15]:
cls = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=3600, per_run_time_limit=420, 
                                                       memory_limit=8000).fit(train_features, train_target)

In [24]:
predict_automl = pd.Series(cls.predict(test_features)).to_csv('sub_automl.csv', index=False)

In [25]:
predict_svm = pd.Series(model.predict(test_features)).to_csv('sub_svm.csv', index=False)